In [ ]:
%load_ext autoreload
%autoreload 2
from infovis21.datamodel.Track import Track
from infovis21.datamodel.User import User
from infovis21.mongodb import MongoAccess as ma
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

In [ ]:
# I know the without is a bit redundant as we could just use the total size of the collection which would be faster but it's also a sanity check for the edge case values of genres

pipeline_with_genre = [
    {'$set': {'n_of_genres': {'$size' : {'$ifNull': [ "$genres", [] ]}} } },
    {'$match': {'n_of_genres': {'$gt': 0}}}
]
res_with_genre = list(ma.coll_artists.aggregate(pipeline_with_genre))
print(f'With genre info in artists {len(res_with_genre)}')

pipeline_without_genre = [
    {'$set': {'n_of_genres': {'$size' : {'$ifNull': [ "$genres", [] ]}} } },
    {'$match': {'n_of_genres': {'$eq': 0}}}
]
res_without_genre = list(ma.coll_artists.aggregate(pipeline_without_genre))
print(f'Without genre info in artists {len(res_without_genre)}')

res_with_genre = list(ma.coll_tracks.aggregate(pipeline_with_genre))
print(f'With genre info in tracks {len(res_with_genre)}')
res_without_genre = list(ma.coll_tracks.aggregate(pipeline_without_genre))
print(f'Without genre info in tracks {len(res_without_genre)}')

In [ ]:
pipeline = [
    {'$unwind' : '$genres'},
    { '$unset': ['_id'] },
]
features = [  
    "danceability",
    "duration_ms",
    "energy",
    "instrumentalness",
    "liveness",
    "loudness",
    "speechiness",
    "tempo",
    "valence",
    "popularity",
    "key",
    "mode",
    "acousticness",
    'explicit',
    'year',
  ]
target = ['genres']
df_artists = pd.DataFrame(ma.coll_artists.aggregate(pipeline))

In [ ]:
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline

# X, y = make_classification(random_state=0)
# X_train, X_test, y_train, y_test = train_test_split(df_artists[features].values, df_artists[target].values.squeeze())
# pipe = Pipeline([('scaler', StandardScaler()), ('GBC', GradientBoostingClassifier())])
# pipe.fit(X_train, y_train)
# pipe.score(X_test, y_test)

In [ ]:
# pipe.score(X_test, y_test)

In [ ]:
### a big share of artists don't have a genre associated with them in the data_w_genres.csv file

d_i = df_artists

In [ ]:
# from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
d_i['genres'] = enc.fit_transform(d_i.genres)

In [ ]:
# replace missing genre label with NaN
d_i.genres = d_i.genres.replace(10840, np.NaN)

# drop strings for imputation
d_i = d_i.drop(['artists', 'popularity', ],axis=1)

In [ ]:
## imputation
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)

# dataframe_new = pd.DataFrame(imputer.fit_transform(dataframe))
d_i = pd.DataFrame(imputer.fit_transform(d_i), columns = d_i.columns)

In [ ]:
# after imputation, values are floats, to be recognised by encoder, we have to revert it to int

d_i.genres = d_i.genres.astype(int)

# to add cat values artists and popularity back in

d_i = pd.concat([d_i, df_artists.artists, df_artists.popularity], axis=1)

In [ ]:
# associate label with genre again

d_i.genres = enc.inverse_transform(d_i.genres)

# d_i.genres.head()